In [ ]:
# 对静态人脸图像文件进行68个特征点的标定
import dlib         # 人脸识别的库 Dlib
import numpy as np,numpy  # 数据处理的库 numpy
import cv2   # 图像处理的库 OpenCv
import sys

In [ ]:
# Dlib 检测器和预测器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

#获取68个面孔特征点，输入函数是图片，返回68个图片的像素点数组，数组index从0开始
def get_landmarks(img):
    faces = detector(img, 1)# 人脸数，第二个参数inclass为1时检测RGB图片，参数为0时检测灰度图片
    if len(faces) == 1:
        return np.matrix([[p.x, p.y] for p in predictor(img, faces[0]).parts()])

In [ ]:
#Procrustes分析方法是对两个形状进行归一化处理 。从数学上来讲，普氏分析就是利用最小二乘法寻找形状A到形状B的仿射变换。
#常规 Procrustes 分析法，旋转缩放移动第一个输入向量点来匹配第二输入个向量点。
#输入参数points1和points2分别是两个图片的输入矩阵，矩阵的每一行都是面部特征点的坐标-像素。
#输出是points1按照points2旋转后得到的矩阵。
#特征点坐标计算列平均值，各坐标减去均值，计算全局标准差，各坐标除以标准差，进行归一化。

def transformation_from_points(points1, points2):
    """
    Return an affine transformation [s * R | T] such that:
        sum ||s*R*p1,i + T - p2,i||^2
    is minimized.
    """
    # Solve the procrustes problem by subtracting centroids, scaling by the
    # standard deviation, and then using the SVD to calculate the rotation. See
    # the following for more details:
    #   https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    # 将特征点转换成matrix
    points1=np.matrixlib.defmatrix.matrix(points1)
    points2=np.matrixlib.defmatrix.matrix(points2)
    points1 = points1.astype(numpy.float64)
    points2 = points2.astype(numpy.float64)
    #计算points1和points2的平均值，axis = 0：压缩行，对各列求均值，返回 1* n 矩阵，axis 不设置值，对 m*n 个数求均值，返回一个实数，axis =1 ：压缩列，对各行求均值，返回 m *1 矩阵
    c1 = numpy.mean(points1, axis=0)
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2
    #计算全局标准差，axis=0计算每一列的标准差，axis=1计算每一行的标准差，缺省计算全局
    s1 = numpy.std(points1)
    s2 = numpy.std(points2)
    points1 /= s1
    points2 /= s2
    #奇异值分解
    U, S, Vt = numpy.linalg.svd(points1.T * points2)

    # The R we seek is in fact the transpose of the one given by U * Vt. This
    # is because the above formulation assumes the matrix goes on the right
    # (with row vectors) where as our solution requires the matrix to be on the
    # left (with column vectors).
    R = (U * Vt).T

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])

#使图形根据旋转角度进行对齐操作
def warp_im(im, M, dshape):
    output_im = numpy.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im


In [ ]:
# 从text文本中读取特征点
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))

    return points


# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask


In [ ]:
#再次对上一步的结果图进行取点，然后运用三角仿射利将被融合图片的脸部轮廓、关键点变形为上面得到的脸部关键点
    src_img = tran_src(src_img, src_points, dst_points, face_area)
    

In [ ]:
#最后一步是将融合后的新图片脸部区域用泊松融合算法贴到模特图上。泊松融合可直接使用opencv提供的函数

def merge_img(src_img, dst_img, dst_matrix, dst_points, k_size=None, mat_multiple=None):
    face_mask = np.zeros(src_img.shape, dtype=src_img.dtype)    
    
    for group in core.OVERLAY_POINTS:
        cv2.fillConvexPoly(face_mask, cv2.convexHull(dst_matrix[group]), (255, 255, 255))
        
    r = cv2.boundingRect(np.float32([dst_points[:core.FACE_END]]))

    center = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2))    
    
    if mat_multiple:
        mat = cv2.getRotationMatrix2D(center, 0, mat_multiple)
        face_mask = cv2.warpAffine(face_mask, mat, (face_mask.shape[1], face_mask.shape[0]))    
    
    if k_size:
        face_mask = cv2.blur(face_mask, k_size, center)    
        
    return cv2.seamlessClone(np.uint8(dst_img), src_img, face_mask, center, cv2.NORMAL_CLONE)


In [ ]:
if __name__=='__main__':
    
    filename1 = '/home/fengchao/桌面/11.png'
    filename2 = '/home/fengchao/桌面/3.png'
    alpha = 0.5 #alpha调节融合程度，数值越小第一张图融合成分越多
    
    # 读取图片
    img1 = cv2.imread(filename1);
    img2 = cv2.imread(filename2);
    
    # 将图片像素值转换成浮点数
    img1 = np.float64(img1)
    img2 = np.float64(img2)

    # 读取两张图片的特征点
    points1 = readPoints(filename1 + '.txt')
    points2 = readPoints(filename2 + '.txt')

    #对图二进行旋转缩放移动第一张图向量点来匹配第一张图向量点。
    M= transformation_from_points(points2, points1)
    #使图像二根据图一脸部进行图片整体旋转
    img2=warp_im(img2, M, img1.shape)
    cv2.imshow("Morphed Face", np.uint8(img2))
    cv2.waitKey(0)
    #再次取特征点
    points2=get_landmarks(img2);
    print(type(points1))
    print(type(points2))
    #创建一个list存放morph后的特征点
    morph_points = []

    #计算alpha权重特征点获取融合特征点
    for i in range(0, len(points1)):
        x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
        y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
        morph_points.append((x,y))
    #为模板面孔赋值用于生成基于图片一的图片空间
    morph_img= np.zeros(img1.shape,dtype = img1.dtype)
    # 从tri.txt文档中读入面部三角
    with open("tri.txt") as file :
        for line in file :
            x,y,z = line.split()
            
            x = int(x)
            y = int(y)
            z = int(z)
            
            t1 = [points1[x], points1[y], points1[z]]
            t2 = [points2[x], points2[y], points2[z]]
            t = [morph_points[x], morph_points[y], morph_points[z]]
            #三角融合算法，对上一步转换后的待融合图片再次取关键点，然后与模特图的关键点一起做三角融合成新的图片
            morphTriangle(img1, img2, morph_img, t1, t2, t, alpha)
    dst_img=morph_img
    
    #再次对上一步的结果图进行取点，然后运用三角仿射将模特图片脸部轮廓、关键点变形成上一步得到的脸部关键点
    #src_img = tran_src(src_img, src_points, morph_points, face_area)
    # Display Result
    #cv2.imshow("Morphed Face", np.uint8(dst_img))
    #cv2.waitKey(0)

In [ ]:

if __name__ == '__main__' :

            

    # Display Result
    #cv2.imshow("Morphed Face", np.uint8(imgMorph))
    #cv2.waitKey(0)
